# Data Cleaning

A partir del dataset de Full MovieLens que recoge metadatos sobre las 45.000 películas de esta plataforma, se realizará un estudio para comprobar la  veracidad de la hipótesis que guía este trabajo: La industria del cine no es rentable. Para ello, es necesario empezar por generar un dataframe con los elementos que nos sean de utilidad. 

Reordenaremos el dataset original y nos quedaremos con las columnas que son de nuestro interés:

## Data Structuring

Importamos las librerías necesarias para este paso.

In [285]:
import pandas as pd
import os
import sys

Se importa el dataset accediendo al directorio donde está guardado. 

In [286]:
# Obtenemos la dirección de la carpeta raiz del proyecto
root_project = os.path.dirname(os.getcwd())
# Adjuntamos esta dirección a las rutas de python para poder importar modulos desde aquí
sys.path.append(root_project)
# Completamos la dirección del archivo csv con el dataset
movies_csv_dir = root_project + '/resources/movies_metadata.csv'

# Leemos el dataset desde el directorio donde se encuentra
movies_csv_df = pd.read_csv(movies_csv_dir, sep= ',')
movies_df = movies_csv_df.copy()

Identificando las columnas se hace una breve pero rápida exploración de los datos que contiene el dataset.

In [287]:
print(list(movies_df.columns), '\n')
print('number of columns --->', len(movies_df.columns))

['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count'] 

number of columns ---> 24


De las 24 columnas del dataset nos interesan solo 11. Generamos una lista ordenada de estas columnas a partir de la cual generaremos el dataframe para el estudio. 

In [288]:
l = ['title', 'release_date', 'original_language', 'genres', 'budget', 'revenue', 'production_countries', 'production_companies', 'runtime', 'vote_average', 'vote_count']

In [289]:
movies_df = movies_df[l]
movies_df.head()

,title,release_date,original_language,genres,budget,revenue,production_countries,production_companies,runtime,vote_average,vote_count
0,Toy Story,1995-10-30,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",30000000,373554033.0,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'name': 'Pixar Animation Studios', 'id': 3}]",81.0,7.7,5415.0
1,Jumanji,1995-12-15,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",65000000,262797249.0,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'name': 'TriStar Pictures', 'id': 559}, {'na...",104.0,6.9,2413.0
2,Grumpier Old Men,1995-12-22,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",0,0.0,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",101.0,6.5,92.0
3,Waiting to Exhale,1995-12-22,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",16000000,81452156.0,"[{'iso_3166_1': 'US', 'name': 'United States o...",[{'name': 'Twentieth Century Fox Film Corporat...,127.0,6.1,34.0
4,Father of the Bride Part II,1995-02-10,en,"[{'id': 35, 'name': 'Comedy'}]",0,76578911.0,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'name': 'Sandollar Productions', 'id': 5842}...",106.0,5.7,173.0


## Column exploration

In [290]:
movies_df.dtypes

title                    object
release_date             object
original_language        object
genres                   object
budget                   object
revenue                 float64
production_countries     object
production_companies     object
runtime                 float64
vote_average            float64
vote_count              float64
dtype: object

Esta observación nos indica el tipo de elementos que contiene cada columna, con lo que podemos identificar que manipulaciones debemos llevar a cabo para que los datos puedan ser utilizados en el estudio. 

A simple vista identificamos los sigientes proplemas:

    - release_date: querremos que esta columna contenga elementos del tipo datetime64

    - budget: esta columna deberá contener valores numéricos para su uso

    - genres, production_countries, production_companies: a la información de estas columnas no es posible             
      acceder de forma directa, será necesario realizar data wrangling.

### Dates to datetime64 type

In [291]:
try:
    pd.to_datetime(movies_df.release_date)
except ValueError:
    print('Given date string not likely a datetime')

Given date string not likely a datetime


Al intentar convertir todos los elementos de release_date a datetime64 aparece el error de arriba. Esto ocurre porque al menos un valor de la columna no aparece en un formato que la librería de pandas pueda convertir al tipo datetime64.  
Debemos indentificar estos elementos:

In [292]:
def not_a_datetime_detector(date_column):
    l = []
    for i, it in enumerate(date_column):
        try:
            pd.to_datetime(it)
        except:
            l.append(i)
    print(l)
    return l

In [293]:
not_a_date_list = not_a_datetime_detector(date_column=movies_df.release_date)

[19730, 29503, 35587]


In [294]:
movies_df.iloc[not_a_date_list]

,title,release_date,original_language,genres,budget,revenue,production_countries,production_companies,runtime,vote_average,vote_count
19730,NaN,1,104.0,"[{'name': 'Carousel Productions', 'id': 11176}...",/ff9qCepilowshEtG2GYWwzt2bs4.jpg,NaN,6.0,False,NaN,NaN,NaN
29503,NaN,12,68.0,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...",/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,NaN,7.0,False,NaN,NaN,NaN
35587,NaN,22,82.0,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...",/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,NaN,4.3,False,NaN,NaN,NaN


Observando los datos para estos elementos, parece razonable eliminarlos del dataset ya que tienen muchos otros datos vacíos y no aportarán utilidad al estudio

In [295]:
movies_df = movies_df.drop(not_a_date_list)

In [296]:
movies_df.release_date = pd.to_datetime(movies_df.release_date)
movies_df.release_date

0       1995-10-30
1       1995-12-15
2       1995-12-22
3       1995-12-22
4       1995-02-10
           ...    
45461          NaT
45462   2011-11-17
45463   2003-08-01
45464   1917-10-21
45465   2017-06-09
Name: release_date, Length: 45463, dtype: datetime64[ns]

### Budget to numerical values

In [297]:
movies_df.budget = pd.to_numeric(movies_df.budget)

### Data wrangling

In [298]:
import json

### - genres

Las columnas genres, production_companies y production_countries tienen los datos en una forma en la que no es posible acceder a ellos. Es necesario minar estos datos.

A través de la librería json transformamos la información de las columnas que estén en este formato (como si fuera un 'str') de forma que se reconozcan los objetos de python, por ejemplo: listas, diccionarios, etc. 

In [299]:
movies_df.genres = movies_df.genres.str.replace("\'", "\"") #Si no se reemplazan '' por "", python no lo                                                                      reconoce como un archivo json

movies_df.genres = movies_df.genres.apply(json.loads)   #cada elemento de la columna es legible por python ahora

Vamos a agrupar los generos de cada película en listas:

In [302]:
generos = set({})

for i, element in enumerate(movies_df.genres):
    genre_list = []
    for dictio in element:
        genre_list.append(dictio['name'])
        generos.add(dictio['name'])
    if i == 0:
        genres_dictio = {i: [genre_list]}
    else:
        genres_dictio[i] = [genre_list]
  
print(f'Número de géros distintos ---> {len(generos)}\n', 'Todos los generos del dataset:\n', generos)
genre_df = pd.DataFrame.from_dict(genres_dictio, orient='index', columns=['Genres'])
genre_df

Número de géros distintos ---> 20
 Todos los generos del dataset:
 {'Foreign', 'Western', 'Drama', 'Horror', 'Music', 'War', 'Science Fiction', 'Family', 'TV Movie', 'Documentary', 'Thriller', 'Action', 'Mystery', 'Adventure', 'Fantasy', 'Romance', 'Comedy', 'Animation', 'History', 'Crime'}


,Genres
0,"[Animation, Comedy, Family]"
1,"[Adventure, Fantasy, Family]"
2,"[Romance, Comedy]"
3,"[Comedy, Drama, Romance]"
4,[Comedy]
...,...
45458,"[Drama, Family]"
45459,[Drama]
45460,"[Action, Drama, Thriller]"
45461,[]


Sustiuimos la columna original de nuestro dataset por la que se ha generado:

In [303]:
movies_df['genres'] = genre_df

Puede ser de utilidad crear un dataframe de la columna genres expandida, de forma que todos los generos correspondan a las columnas:

        -se crea un dataframe de una columna compuesta por diccionarios donde la clave y el valor
         corresponden a los generos de cada película

In [304]:
genres_df_dict = pd.DataFrame.from_dict(genre_df.Genres.apply(lambda x: {gen : gen for gen in x}))
genres_df_dict.head(3)

,Genres
0,"{'Animation': 'Animation', 'Comedy': 'Comedy',..."
1,"{'Adventure': 'Adventure', 'Fantasy': 'Fantasy..."
2,"{'Romance': 'Romance', 'Comedy': 'Comedy'}"


        -a partir de este dataframe generamos el dataframe expandido donde cada columna representa uno 
         de los 20 géneros de nuestro dataset original

In [307]:
genre_df_expanded = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in genres_df_dict.Genres.items() ])).transpose()

In [308]:
genre_df_expanded.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,NaN,NaN,Animation,Comedy,NaN,NaN,NaN,Family,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Adventure,NaN,NaN,NaN,NaN,NaN,Family,Fantasy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Comedy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Romance,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Comedy,NaN,NaN,Drama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Romance,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Comedy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


genre_df_expanded.count()

### - production_countries

In [333]:
def json_prep(serie):
    ''' 
    Return a pd.Series object with json elements as a pd.Series with python elements
    '''
    #If '' aren't replaced for "", python won't recognise the json file  
    serie = serie.str.replace("\'", "\"")

    #now every element of the series is legible by python   
    serie = serie.apply(json.loads)                  
    return serie

In [417]:
x = movies_df.production_companies.str.replace("\'", "\"")
s = pd.Series()
l = []

for el in range(0,len(movies_df)):
    try:
        s[el] = json.loads(x[el])
    except:
        l.append(el)

In [420]:
dict1 = movies_df.production_countries[0]
json.loads(dict1)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)

In [359]:
len(movies_df)

45463

In [353]:
def series_dict_to_series_list(serie, use_key, inplace=False):
    '''
    Args:
        serie (pd.Series)
        use_key (str): key name to select from series' dicts
        inplace (bool): bool to indicate if serie is to be replaced by the new series


    From serie (pd.Series of dicts) return the useful information from the dict (use_key) as in list form 
    '''
    serie = json_prep(serie)

    uniq_vals = set({})

    for i, element in enumerate(serie):
        val_list = []
        for dictio in element:
            val_list.append(dictio[use_key])
            uniq_vals.add(dictio[use_key])
        if i == 0:
            serie_dictio = {i: [val_list]}
        else:
            serie_dictio[i] = [val_list]

    print(f'Número de elemenos únicos ---> {len(uniq_vals)}\n', 'Todos los elementos únicos de la columna:\n',         uniq_vals)
    serie_df = pd.DataFrame.from_dict(serie_dictio, orient='index', columns=[serie.name])
    if inplace == True:
        serie = serie_df
    else:
        return genre_df

In [338]:
series_dict_to_series_list(serie=movies_df.production_countries, use_key='name')

JSONDecodeError: Expecting ',' delimiter: line 1 column 39 (char 38)

In [ ]:

generos = set({})

for i, element in enumerate(movies_df.genres):
    genre_list = []
    for dictio in element:
        genre_list.append(dictio['name'])
        generos.add(dictio['name'])
    if i == 0:
        genres_dictio = {i: [genre_list]}
    else:
        genres_dictio[i] = [genre_list]
  
print(f'Número de elemenos únicos ---> {len(generos)}\n', 'Todos los elementos únicos de la columna:\n', generos)
genre_df = pd.DataFrame.from_dict(genres_dictio, orient='index', columns=['Genres'])
genre_df

In [ ]:
genres_df_dict = pd.DataFrame.from_dict(genre_df.Genres.apply(lambda x: {gen : gen for gen in x}))
genres_df_dict.head(3)

genre_df_expanded = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in genres_df_dict.Genres.items() ])).transpose()

In [ ]:
## Data Exploration